In [ ]:
!pip install hmmlearn biopython

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
df = pd.read_csv("/content/CoV-AbDab_080224.csv")

In [ ]:
#create training labels
def label_neutralization(row):
    neut = str(row["Neutralising Vs"]).strip()
    non = str(row["Not Neutralising Vs"]).strip()

    if neut not in ["", "nan", "NaN"] and neut != "None":
        return 1
    if non not in ["", "nan", "NaN"] and non != "None":
        return 0
    return None   # unknown

df["label"] = df.apply(label_neutralization, axis=1)
df = df.dropna(subset=["label"])
df["label"] = df["label"].astype(int)

In [ ]:
#extracting the CDRH3 sequences
df = df[df["CDRH3"].notna()] #remove rows with missing CDRH3
neutralizing_seqs = df[df["label"] == 1]["CDRH3"].tolist() #looks at label =1 (neutralizing), remove rows with CDRH3, makes it into list
non_neut_seqs = df[df["label"] == 0]["CDRH3"].tolist() #looks at label =0 (non- neutralizing), remove rows CDRH3, makes it into list

In [ ]:
#converting amino acid sequences to integers
import numpy as np

AA = "ACDEFGHIKLMNPQRSTVWY"
aa_to_idx = {aa:i for i,aa in enumerate(AA)}

def one_hot(seq):
    arr = np.zeros((len(seq), 20), dtype=int)
    for i, aa in enumerate(seq):
        if aa in aa_to_idx:
            arr[i, aa_to_idx[aa]] = 1
    return arr


In [ ]:
neut_encoded = [one_hot(s) for s in neutralizing_seqs]
non_neut_encoded = [one_hot(s) for s in non_neut_seqs]

In [ ]:
#default trained for 10 iterations
from hmmlearn.hmm import MultinomialHMM

np.random.seed(42)  # Added this for reproducibility

def train_hmm(seqs, n_states=6):
    model = MultinomialHMM(n_components=n_states, random_state=42) #take out random state if results are worse
    model.n_features = 20  # 20 amino acids

    lengths = [len(s) for s in seqs]
    X = np.vstack(seqs)  # shape = (total_len, 20)

    model.fit(X, lengths)
    return model


In [ ]:
HMM_neut = train_hmm(neut_encoded, n_states=6)
HMM_non = train_hmm(non_neut_encoded, n_states=6)

In [ ]:
import pandas as pd
import numpy as np

# List of amino acids
AA = "ACDEFGHIKLMNPQRSTVWY"
aa_to_idx = {aa:i for i,aa in enumerate(AA)}

# Function to create labeled DataFrame for emission matrix
def create_emission_df(emission_matrix, amino_acids=AA):
    n_states = emission_matrix.shape[0]
    df = pd.DataFrame(emission_matrix, columns=list(amino_acids))
    df.index = [f"State_{i+1}" for i in range(n_states)]
    return df

# Create labeled DataFrames
em_neut_df = create_emission_df(HMM_neut.emissionprob_)
em_non_df = create_emission_df(HMM_non.emissionprob_)

# Display the DataFrames
print("Neutralizing Antibodies Emission Probability Matrix:")
display(em_neut_df)

print("\nNon-Neutralizing Antibodies Emission Probability Matrix:")
display(em_non_df)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    roc_auc_score,
    roc_curve,
    confusion_matrix,
    classification_report,
    balanced_accuracy_score,
)

# ============================================================================
# SECTION 1: BIOCHEMICAL FEATURE EXTRACTION
# ============================================================================

class BiochemicalFeatureExtractor:
    """Extract biochemical descriptors from CDRH3 sequences"""

    HYDROPHOBICITY = {
        'A': 1.8, 'R': -4.5, 'N': -3.5, 'D': -3.5, 'C': 2.5,
        'Q': -3.5, 'E': -3.5, 'G': -0.4, 'H': -3.2, 'I': 4.5,
        'L': 3.8, 'K': -3.9, 'M': 1.9, 'F': 2.8, 'P': -1.6,
        'S': -0.8, 'T': -0.7, 'W': -0.9, 'Y': -1.3, 'V': 4.2
    }

    CHARGE = {
        'A': 0, 'R': 1, 'N': 0, 'D': -1, 'C': 0,
        'Q': 0, 'E': -1, 'G': 0, 'H': 0.5, 'I': 0,
        'L': 0, 'K': 1, 'M': 0, 'F': 0, 'P': 0,
        'S': 0, 'T': 0, 'W': 0, 'Y': 0, 'V': 0
    }

    AROMATIC = {'F', 'W', 'Y', 'H'}
    POLAR = {'S', 'T', 'N', 'Q', 'C'}
    ALIPHATIC = {'A', 'V', 'I', 'L', 'M'}

    MOLECULAR_WEIGHT = {
        'A': 89.1, 'R': 174.2, 'N': 132.1, 'D': 133.1, 'C': 121.2,
        'Q': 146.2, 'E': 147.1, 'G': 75.1, 'H': 155.2, 'I': 131.2,
        'L': 131.2, 'K': 146.2, 'M': 149.2, 'F': 165.2, 'P': 115.1,
        'S': 105.1, 'T': 119.1, 'W': 204.2, 'Y': 181.2, 'V': 117.1
    }

    def __init__(self):
        self.feature_names = []

    def extract_features(self, sequence):
        features = {}

        features['length'] = len(sequence)
        hydros = [self.HYDROPHOBICITY.get(aa, 0) for aa in sequence]
        charges = [self.CHARGE.get(aa, 0) for aa in sequence]

        features['hydrophobicity_mean'] = np.mean(hydros)
        features['hydrophobicity_std'] = np.std(hydros)
        features['charge_total'] = sum(charges)
        features['charge_mean'] = features['charge_total'] / len(sequence)

        features['aromatic_count'] = sum(1 for aa in sequence if aa in self.AROMATIC)
        features['aromatic_fraction'] = features['aromatic_count'] / len(sequence)

        features['polar_count'] = sum(1 for aa in sequence if aa in self.POLAR)
        features['polar_fraction'] = features['polar_count'] / len(sequence)

        features['aliphatic_count'] = sum(1 for aa in sequence if aa in self.ALIPHATIC)
        features['aliphatic_fraction'] = features['aliphatic_count'] / len(sequence)

        features['molecular_weight'] = sum([self.MOLECULAR_WEIGHT.get(aa, 0) for aa in sequence])
        features['avg_residue_weight'] = features['molecular_weight'] / len(sequence)

        n_third = len(sequence) // 3
        if n_third > 0:
            n_term = sequence[:n_third]
            c_term = sequence[-n_third:]
            middle = sequence[n_third:-n_third] if len(sequence) > 2 * n_third else sequence

            features['n_term_hydrophobicity'] = np.mean([self.HYDROPHOBICITY.get(aa, 0) for aa in n_term])
            features['c_term_hydrophobicity'] = np.mean([self.HYDROPHOBICITY.get(aa, 0) for aa in c_term])
            features['middle_hydrophobicity'] = np.mean(
                [self.HYDROPHOBICITY.get(aa, 0) for aa in middle]
            ) if middle else 0

            features['n_term_charge'] = sum([self.CHARGE.get(aa, 0) for aa in n_term]) / len(n_term)
            features['c_term_charge'] = sum([self.CHARGE.get(aa, 0) for aa in c_term]) / len(c_term)
        else:
            features['n_term_hydrophobicity'] = features['hydrophobicity_mean']
            features['c_term_hydrophobicity'] = features['hydrophobicity_mean']
            features['middle_hydrophobicity'] = features['hydrophobicity_mean']
            features['n_term_charge'] = features['charge_mean']
            features['c_term_charge'] = features['charge_mean']

        return features

    def extract_batch(self, sequences):
        feature_list = [self.extract_features(seq) for seq in sequences]
        df = pd.DataFrame(feature_list)
        self.feature_names = df.columns.tolist()
        return df

# ============================================================================
# SECTION 2: HMM MOTIF PROBABILITY EXTRACTION
# ============================================================================

def extract_hmm_emission_features(sequences, emission_probs_neut, emission_probs_non_neut,
                                  amino_acids='ACDEFGHIKLMNPQRSTVWY'):
    aa_to_idx = {aa: i for i, aa in enumerate(amino_acids)}
    n_states = emission_probs_neut.shape[0]

    motif_features = []

    for seq in sequences:
        features = {}

        for state_idx in range(n_states):
            neut_emissions = []
            non_neut_emissions = []

            for aa in seq:
                if aa in aa_to_idx:
                    aa_idx = aa_to_idx[aa]
                    neut_emissions.append(emission_probs_neut[state_idx, aa_idx])
                    non_neut_emissions.append(emission_probs_non_neut[state_idx, aa_idx])

            if neut_emissions:
                features[f'state_{state_idx}_neut_emission_mean'] = np.mean(neut_emissions)
                features[f'state_{state_idx}_non_neut_emission_mean'] = np.mean(non_neut_emissions)
                features[f'state_{state_idx}_emission_ratio'] = (
                    np.mean(neut_emissions) / (np.mean(non_neut_emissions) + 1e-10)
                )
                features[f'state_{state_idx}_neut_emission_max'] = np.max(neut_emissions)
                features[f'state_{state_idx}_non_neut_emission_max'] = np.max(non_neut_emissions)

        log_prob_neut = 0.0
        log_prob_non_neut = 0.0

        for aa in seq:
            if aa in aa_to_idx:
                aa_idx = aa_to_idx[aa]
                log_prob_neut += np.log(np.max(emission_probs_neut[:, aa_idx]) + 1e-300)
                log_prob_non_neut += np.log(np.max(emission_probs_non_neut[:, aa_idx]) + 1e-300)

        features['log_likelihood_neut'] = log_prob_neut
        features['log_likelihood_non_neut'] = log_prob_non_neut
        features['log_likelihood_ratio'] = log_prob_neut - log_prob_non_neut

        motif_features.append(features)

    return pd.DataFrame(motif_features)

# ============================================================================
# SECTION 3: INTEGRATED PIPELINE
# ============================================================================

def add_interaction_features(X,
                             biochem_cols=None,
                             motif_cols=None,
                             max_motif=10):
    X = X.copy()

    if biochem_cols is None:
        biochem_cols = [
            c for c in X.columns
            if any(k in c for k in [
                'hydrophobicity', 'charge', 'aromatic_fraction',
                'polar_fraction', 'aliphatic_fraction', 'length'
            ])
        ]

    if motif_cols is None:
        all_motif = [c for c in X.columns
                     if c.startswith('state_') or 'log_likelihood' in c]
        motif_cols = all_motif[:max_motif]

    print(f"Adding interactions between {len(biochem_cols)} biochem "
          f"and {len(motif_cols)} motif features")

    inter_cols = {}
    for b in biochem_cols:
        for m in motif_cols:
            inter_name = f'{b}__X__{m}'
            inter_cols[inter_name] = X[b] * X[m]

    inter_df = pd.DataFrame(inter_cols, index=X.index)
    X = pd.concat([X, inter_df], axis=1)

    print(f"Total features after interactions: {X.shape[1]}")
    return X


def create_integrated_feature_matrix(sequences, labels, emission_probs_neut, emission_probs_non_neut):
    print("Extracting HMM emission-based features...")
    motif_features = extract_hmm_emission_features(
        sequences, emission_probs_neut, emission_probs_non_neut
    )

    print("Extracting biochemical features...")
    extractor = BiochemicalFeatureExtractor()
    biochem_features = extractor.extract_batch(sequences)

    X = pd.concat([motif_features, biochem_features], axis=1)
    y = np.array(labels)

    X = add_interaction_features(X)

    feature_names = X.columns.tolist()

    print(f"Total features: {len(feature_names)}")
    print(f"  - Motif features: {len(motif_features.columns)}")
    print(f"  - Biochemical features: {len(biochem_features.columns)}")

    return X, y, feature_names

# ============================================================================
# SECTION 4: BAYESIAN MODEL USING MAP ESTIMATION (NO CV, NO HESSIAN)
# ============================================================================

class FastBayesianNeutralizationModel:
    """
    Logistic regression with L2 regularization (Gaussian prior).
    No CV, no Hessian; fast and stable.
    """

    def __init__(self, prior_scale=2.5):
        self.prior_scale = prior_scale
        self.lambda_reg = 1.0 / (prior_scale ** 2)
        self.coef_ = None
        self.intercept_ = None
        self.feature_names = None
        self.scaler = StandardScaler()
        self.std_errors_ = None
        self.best_thresh_ = None

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))

    def _negative_log_posterior(self, params, X, y):
        intercept = params[0]
        coef = params[1:]

        z = intercept + X @ coef
        p = self._sigmoid(z)

        epsilon = 1e-15
        p = np.clip(p, epsilon, 1 - epsilon)

        N = len(y)
        N_pos = np.sum(y == 1)
        N_neg = N - N_pos
        if N_pos == 0 or N_neg == 0:
            w_pos = 1.0
            w_neg = 1.0
        else:
            w_pos = N / (2.0 * N_pos)
            w_neg = N / (2.0 * N_neg)

        nll = -np.sum(
            w_pos * y * np.log(p) +
            w_neg * (1 - y) * np.log(1 - p)
        )

        nlp = 0.5 * self.lambda_reg * np.sum(coef ** 2)

        return nll + nlp

    def fit(self, X, y, feature_names):
        self.feature_names = feature_names

        X_scaled = self.scaler.fit_transform(X)
        n_features = X_scaled.shape[1]
        init_params = np.zeros(n_features + 1)

        print("Fitting model using MAP estimation...")
        print(f"Prior scale (σ): {self.prior_scale}")
        print(f"Regularization strength (λ): {self.lambda_reg:.4f}")

        result = minimize(
            self._negative_log_posterior,
            init_params,
            args=(X_scaled, y),
            method='L-BFGS-B',
            options={'maxiter': 1000}
        )

        if result.success:
            print("Optimization converged successfully!")
        else:
            print("Warning: Optimization may not have fully converged")

        self.intercept_ = result.x[0]
        self.coef_ = result.x[1:]

        # Dummy SDs so coefficient summary still works
        self.std_errors_ = np.ones_like(result.x) * 0.01

        return self

    def predict_proba(self, X):
        X_scaled = self.scaler.transform(X)
        z = self.intercept_ + X_scaled @ self.coef_
        prob = self._sigmoid(z)

        if self.std_errors_ is not None:
            z_std = np.sqrt(np.sum((X_scaled * self.std_errors_[1:]) ** 2, axis=1))
            prob_lower = self._sigmoid(z - 1.96 * z_std)
            prob_upper = self._sigmoid(z + 1.96 * z_std)
        else:
            prob_lower = np.full_like(prob, np.nan)
            prob_upper = np.full_like(prob, np.nan)

        return prob, prob_lower, prob_upper

    def get_coefficient_summary(self):
        feature_list = ['alpha'] + self.feature_names
        summary = pd.DataFrame({
            'feature': feature_list,
            'mean': np.concatenate([[self.intercept_], self.coef_]),
            'sd': self.std_errors_,
        })
        summary['ci_lower'] = summary['mean'] - 1.96 * summary['sd']
        summary['ci_upper'] = summary['mean'] + 1.96 * summary['sd']
        return summary

    def plot_coefficient_forest(self, top_n=20):
        coef_summary = self.get_coefficient_summary()
        coef_summary = coef_summary[coef_summary['feature'] != 'alpha']
        coef_summary['abs_mean'] = np.abs(coef_summary['mean'])
        coef_summary = coef_summary.sort_values('abs_mean', ascending=False).head(top_n)

        fig, ax = plt.subplots(figsize=(10, 8))
        y_pos = np.arange(len(coef_summary))
        ax.errorbar(
            coef_summary['mean'],
            y_pos,
            xerr=[coef_summary['mean'] - coef_summary['ci_lower'],
                  coef_summary['ci_upper'] - coef_summary['mean']],
            fmt='o',
            capsize=5,
            capthick=2
        )

        ax.axvline(0, linestyle='--', alpha=0.5)
        ax.set_yticks(y_pos)
        ax.set_yticklabels(coef_summary['feature'])
        ax.set_xlabel('Coefficient Value (Standardized)', fontsize=12)
        ax.set_title(f'Top {top_n} Features by Coefficient Magnitude', fontsize=14)
        ax.grid(axis='x', alpha=0.3)
        plt.tight_layout()
        return fig

# ============================================================================
# SECTION 5: MODEL EVALUATION
# ============================================================================

def evaluate_model(model, X_test, y_test):
    print("\n" + "=" * 60)
    print("MODEL EVALUATION")
    print("=" * 60)

    y_pred_proba, _, _ = model.predict_proba(X_test)

    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    youden_j = tpr - fpr
    best_idx = np.argmax(youden_j)
    best_thresh = thresholds[best_idx]
    print(f"\nOptimal threshold (Youden J): {best_thresh:.3f}")

    model.best_thresh_ = best_thresh
    y_pred = (y_pred_proba >= best_thresh).astype(int)

    auc = roc_auc_score(y_test, y_pred_proba)
    print(f"\nROC-AUC Score: {auc:.4f}")

    bal_acc = balanced_accuracy_score(y_test, y_pred)
    print(f"Balanced Accuracy: {bal_acc:.4f}")

    print("\nClassification Report:")
    print(classification_report(
        y_test,
        y_pred,
        target_names=['Non-Neutralizing', 'Neutralizing']
    ))

    cm = confusion_matrix(y_test, y_pred)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax1)
    ax1.set_xlabel('Predicted')
    ax1.set_ylabel('Actual')
    ax1.set_title('Confusion Matrix')
    ax1.set_xticklabels(['Non-Neut', 'Neut'])
    ax1.set_yticklabels(['Non-Neut', 'Neut'])

    ax2.plot(fpr, tpr, linewidth=2, label=f'ROC (AUC = {auc:.3f})')
    ax2.plot([0, 1], [0, 1], 'k--', label='Random')
    ax2.set_xlabel('False Positive Rate')
    ax2.set_ylabel('True Positive Rate')
    ax2.set_title('ROC Curve')
    ax2.legend()
    ax2.grid(alpha=0.3)

    plt.tight_layout()
    plt.show()

    return auc, y_pred_proba, best_thresh

# ============================================================================
# SECTION 6: MAIN EXECUTION (NO PRIOR TUNING)
# ============================================================================

def main_example_fast(emission_probs_neut, emission_probs_non_neut, sequences, labels):
    print("=" * 60)
    print("BAYESIAN LOGISTIC REGRESSION (MAP ESTIMATION)")
    print("=" * 60)

    print(f"\n Loaded emission matrices:")
    print(f"  - Neutralizing HMM: {emission_probs_neut.shape}")
    print(f"  - Non-neutralizing HMM: {emission_probs_non_neut.shape}")
    print(f"\n Data summary:")
    print(f"  - Total sequences: {len(sequences)}")
    print(f"  - Neutralizing: {sum(labels == 1)}")
    print(f"  - Non-neutralizing: {sum(labels == 0)}")

    print("\n" + "=" * 60)
    print("EXTRACTING FEATURES")
    print("=" * 60 + "\n")

    X_full, y, feature_names = create_integrated_feature_matrix(
        sequences, labels, emission_probs_neut, emission_probs_non_neut
    )

    X_train, X_test, y_train, y_test = train_test_split(
        X_full.values, y, test_size=0.2, random_state=42, stratify=y
    )

    print(f"\nTraining set: {len(X_train)} samples")
    print(f"Test set: {len(X_test)} samples")

    import time
    start_time = time.time()

    # Single fit, fixed prior_scale
    model = FastBayesianNeutralizationModel(prior_scale=2.5)
    model.fit(X_train, y_train, feature_names)

    elapsed_time = time.time() - start_time
    print(f"\n Training completed in {elapsed_time:.1f} seconds!")

    auc, y_pred_proba, best_thresh = evaluate_model(model, X_test, y_test)

    print("\n" + "=" * 60)
    print("COEFFICIENT SUMMARY (Top 15)")
    print("=" * 60)
    coef_summary = model.get_coefficient_summary()
    print(coef_summary.head(15).to_string(index=False))

    print("\n" + "=" * 60)
    print("GENERATING VISUALIZATIONS")
    print("=" * 60)
    model.plot_coefficient_forest(top_n=15)
    plt.show()

    # If you have plot_prediction_uncertainty defined elsewhere, you can still call it:
    try:
        y_pred_mean, y_pred_lower, y_pred_upper = model.predict_proba(X_test)
        plot_prediction_uncertainty(y_test, y_pred_mean, y_pred_lower, y_pred_upper)
    except NameError:
        pass

    print("\n Fast analysis complete!")
    print("\nKey outputs:")
    print("- MAP coefficient estimates")
    print("- Feature importance rankings")
    print("- Model performance metrics")

    return model, coef_summary, X_full, feature_names

print("Bayesian model loaded!")

In [ ]:
# ==============================================================================
# FAST AIM 2: RUN BAYESIAN LOGISTIC REGRESSION (MAP ESTIMATION)
# ==============================================================================
# This runs in 1-2 minutes instead of 15+ minutes!
# ==============================================================================

print("="*70)
print("PREPARING DATA FROM AIM 1")
print("="*70)

# Extract emission probability matrices from your trained HMMs
emission_probs_neut = HMM_neut.emissionprob_
emission_probs_non_neut = HMM_non.emissionprob_

# Combine sequences and create labels
all_sequences = neutralizing_seqs + non_neut_seqs
all_labels = np.array([1]*len(neutralizing_seqs) + [0]*len(non_neut_seqs))

print(f"\n Loaded neutralizing HMM emissions: {emission_probs_neut.shape}")
print(f" Loaded non-neutralizing HMM emissions: {emission_probs_non_neut.shape}")
print(f" Total sequences: {len(all_sequences)}")
print(f"  - Neutralizing: {sum(all_labels == 1)}")
print(f"  - Non-neutralizing: {sum(all_labels == 0)}")

# ==============================================================================
# RUN THE FAST ANALYSIS (uses MAP instead of MCMC)
# ==============================================================================

print("\n" + "="*70)
print("RUNNING FAST AIM 2 ANALYSIS")
print("="*70)

# Run the fast analysis
model, coef_summary, X_full, feature_names = main_example_fast(
    emission_probs_neut,
    emission_probs_non_neut,
    all_sequences,
    all_labels
)

# ==============================================================================
# FEATURE IMPORTANCE BY CATEGORY
# ==============================================================================

print("\n" + "="*70)
print("FEATURE IMPORTANCE BY CATEGORY")
print("="*70)

# Get all features
all_features = coef_summary[coef_summary['feature'] != 'alpha'].copy()
all_features['abs_mean'] = np.abs(all_features['mean'])

# Categorize features
motif_features = all_features[all_features['feature'].str.contains('state_|log_likelihood')]
biochem_features = all_features[~all_features['feature'].str.contains('state_|log_likelihood')]

print(f"\n TOP 10 HMM MOTIF FEATURES:")
print("-" * 70)
for idx, row in motif_features.nlargest(10, 'abs_mean').iterrows():
    direction = "↑" if row['mean'] > 0 else "↓"
    print(f"{direction} {row['feature']:45s} β={row['mean']:7.3f}  95% CI: [{row['ci_lower']:6.3f}, {row['ci_upper']:6.3f}]")

print(f"\n TOP 10 BIOCHEMICAL FEATURES:")
print("-" * 70)
for idx, row in biochem_features.nlargest(10, 'abs_mean').iterrows():
    direction = "↑" if row['mean'] > 0 else "↓"
    print(f"{direction} {row['feature']:45s} β={row['mean']:7.3f}  95% CI: [{row['ci_lower']:6.3f}, {row['ci_upper']:6.3f}]")

# ==============================================================================
# SUMMARY
# ==============================================================================

print("\n" + "="*70)
print("FAST ANALYSIS COMPLETE!")
print("="*70)

print(f"\n Summary:")
print(f"  - Total features analyzed: {len(all_features)}")
print(f"    • HMM motif features: {len(motif_features)}")
print(f"    • Biochemical features: {len(biochem_features)}")

print(f"\n Top 3 Most Influential Features Overall:")
top_3 = all_features.nlargest(3, 'abs_mean')
for i, (idx, row) in enumerate(top_3.iterrows(), 1):
    effect = "promotes" if row['mean'] > 0 else "inhibits"
    print(f"  {i}. {row['feature']}")
    print(f"     → {effect} neutralization (β = {row['mean']:.3f}, 95% CI: [{row['ci_lower']:.3f}, {row['ci_upper']:.3f}])")

print("\n Aim 2 Complete!")
print("\nNote: This uses MAP estimation (fast) instead of full MCMC (slow)")
print("Results are very similar but computed 50-100x faster!")
print("\nResults stored in variables:")
print("  - model: trained FastBayesianNeutralizationModel")

print("  - coef_summary: DataFrame with coefficient estimates")

In [ ]:
import pymc as pm
import re
# --------------------------
# 1. Extract V-gene family
# --------------------------

def extract_v_family_column(df, v_col=None):
    """
    Add a 'V_gene_family' column to df based on a V-gene column.
    For CoV-AbDab, we primarily use 'Heavy V Gene'.
    """
    if "V_gene_family" in df.columns:
        return df

    df = df.copy()

    if v_col is None:
        candidate_cols = [
            "Heavy V Gene", "VHorVHH", "V_gene", "v_gene",
            "V_CALL", "v_call", "heavy_v_gene", "VH_gene", "vh_gene"
        ]
        found = [c for c in candidate_cols if c in df.columns]
        if not found:
            raise ValueError(
                "No V-gene column found. "
                "Provide v_col or ensure one of the standard names exists."
            )
        v_col = found[0]

    def parse_family(x):
        if pd.isna(x):
            return np.nan

        s = str(x).strip()

        if "(" in s:
            s = s.split("(")[0].strip()

        if "*" in s:
            s = s.split("*")[0].strip()

        s = s.split()[0]

        m = re.match(r"^([A-Za-z]+[0-9]+)", s)
        if m:
            return m.group(1)

        if "-" in s:
            return s.split("-")[0].strip() or np.nan

        return s or np.nan

    df["V_gene_family"] = df[v_col].apply(parse_family)
    return df

def run_aim3_MAP(df, X_features, y_labels, feature_names, min_group_size=15):
    """
    Hierarchical Bayesian logistic regression with V-gene family grouping.

    - Group-specific intercepts a_g ~ N(mu_a, sigma_a^2)
    - Global slopes beta_f shared across families
    """

    # 1. Alignment checks
    assert len(df) == len(X_features) == len(y_labels), "Data must be aligned."
    df = df.copy()

    # 2. Extract V-gene family
    df = extract_v_family_column(df)

    # Remove NA families
    mask_valid = df["V_gene_family"].notna()
    df = df[mask_valid].reset_index(drop=True)
    X_features = X_features[mask_valid]
    y_labels = np.array(y_labels)[mask_valid]

    # 3. Filter by minimum group size
    family_counts = df["V_gene_family"].value_counts()
    valid = family_counts[family_counts >= min_group_size].index
    keep_mask = df["V_gene_family"].isin(valid)
    df = df[keep_mask].reset_index(drop=True)
    X_features = X_features[keep_mask]
    y_labels = np.array(y_labels)[keep_mask]

    print(f"Retained {len(df)} samples across {len(valid)} families")

    # 4. Prepare group codes
    df["V_gene_family"] = df["V_gene_family"].astype("category")
    group_idx = df["V_gene_family"].cat.codes.values
    group_names = df["V_gene_family"].cat.categories.tolist()
    n_groups = len(group_names)

    # 5. Train-test split
    X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
        X_features,
        y_labels,
        group_idx,
        test_size=0.2,
        random_state=42,
        stratify=y_labels
    )

    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s = scaler.transform(X_test)

    n_features = X_train_s.shape[1]

    # Checkpoint to ensure dataset was pre-processed correctly
    print("Any NaNs in X_train_s?", np.isnan(X_train_s).any())
    print("Any infs in X_train_s?", np.isinf(X_train_s).any())
    print("X_train_s min/max:", X_train_s.min(), X_train_s.max())

    # 6. Hierarchical logistic regression: random intercepts + global slopes
    with pm.Model() as hierarchical_model:
        # Hyperpriors for intercepts
        mu_a = pm.Normal("mu_a", mu=0, sigma=2)
        sigma_a = pm.HalfNormal("sigma_a", sigma=2)

        # Group-specific intercepts
        a = pm.Normal("a", mu=mu_a, sigma=sigma_a, shape=n_groups)

        # Global slopes (no hierarchy on slopes)
        beta = pm.Normal("beta", mu=0, sigma=2, shape=n_features)

        # Linear predictor
        logit_p = a[group_train] + pm.math.dot(X_train_s, beta)

        # Likelihood
        y_obs = pm.Bernoulli("y_obs", logit_p=logit_p, observed=y_train)

        # MAP estimate
        map_estimate = pm.find_MAP(method="Powell")

    # 7. Predictions
    a_map = map_estimate["a"]        # (n_groups,)
    beta_map = map_estimate["beta"]  # (n_features,)

    def predict(X, groups):
        logit = a_map[groups] + np.dot(X, beta_map)
        return 1 / (1 + np.exp(-logit))

    y_train_pred = predict(X_train_s, group_train)
    y_test_pred = predict(X_test_s, group_test)

    # 8. Metrics + plots (same style as Aim 2)
    fpr, tpr, thresholds = roc_curve(y_test, y_test_pred)
    youden_j = tpr - fpr
    best_idx = np.argmax(youden_j)
    best_thresh = thresholds[best_idx]
    print(f"\nOptimal threshold (Youden J): {best_thresh:.3f}")

    y_test_bin = (y_test_pred >= best_thresh).astype(int)

    auc_train = roc_auc_score(y_train, y_train_pred)
    auc_test = roc_auc_score(y_test, y_test_pred)
    bal_acc = balanced_accuracy_score(y_test, y_test_bin)

    print(f"Train AUC: {auc_train:.3f}, Test AUC: {auc_test:.3f}")
    print(f"Balanced accuracy (test): {bal_acc:.3f}")
    report_test = classification_report(
        y_test, y_test_bin,
        target_names=['Non-Neutralizing', 'Neutralizing']
    )
    print("Test Classification Report:\n", report_test)

    # Confusion matrix + ROC curve
    cm = confusion_matrix(y_test, y_test_bin)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax1)
    ax1.set_xlabel('Predicted')
    ax1.set_ylabel('Actual')
    ax1.set_title('Aim 3 Confusion Matrix')
    ax1.set_xticklabels(['Non-Neut', 'Neut'])
    ax1.set_yticklabels(['Non-Neut', 'Neut'])

    ax2.plot(fpr, tpr, linewidth=2, label=f'ROC (AUC = {auc_test:.3f})')
    ax2.plot([0, 1], [0, 1], 'k--', label='Random')
    ax2.set_xlabel('False Positive Rate')
    ax2.set_ylabel('True Positive Rate')
    ax2.set_title('Aim 3 ROC Curve')
    ax2.legend()
    ax2.grid(alpha=0.3)

    plt.tight_layout()
    plt.show()

    return {
        "model": hierarchical_model,
        "map_estimate": map_estimate,
        "scaler": scaler,
        "y_train_pred": y_train_pred,
        "y_test_pred": y_test_pred,
        "auc_train": auc_train,
        "auc_test": auc_test,
        "balanced_acc": bal_acc,
        "best_thresh": best_thresh,
        "report_test": report_test,
        "group_names": group_names,
        "fpr": fpr,
        "tpr": tpr,
    }

In [ ]:
# Making copies to be used in Aim 3 so that results don't overwrite Aim 2
df_neut = df[df["label"] == 1].copy()
df_non  = df[df["label"] == 0].copy()
df_aim3 = pd.concat([df_neut, df_non], ignore_index=True)

# Checking to make sure dimensions match
print(df_aim3.shape[0], X_full.shape[0], len(all_labels))

In [ ]:
cols_aim3 = [c for c in X_full.columns if "__X__" not in c]   # no interaction terms
X_aim3 = X_full[cols_aim3].values
feature_names_aim3 = cols_aim3

In [ ]:
from sklearn.feature_selection import VarianceThreshold

vt = VarianceThreshold(threshold=1e-4)
X_aim3 = vt.fit_transform(X_aim3)
feature_names_aim3 = [cols_aim3[i] for i, v in enumerate(vt.variances_) if v > 1e-4]

In [ ]:
results = run_aim3_MAP(
    df=df_aim3,                  # DataFrame with Heavy V Gene etc.
    X_features=X_aim3,      # 2D array (n_samples, n_features)
    y_labels=all_labels,             # 1D array-like of 0/1
    feature_names=feature_names_aim3,
    min_group_size=15
)